In [ ]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime, read, Stream
import os 
import errno
# list of stations
seislist=['R1FC6','R6DC2','R05EA','R2196','S5DF3','R10FB','R02CF','RDD69','RD48B','REC3A','RD569','RCC9B','R7AA4','R32F0'] # COD SEISMOMETRE
locations=['Mihai Eminescu Satu Mare','Mihai Viteazu Ploiesti','Gheorghe Lazar Bucuresti',
           'Costache Negruzzi Iasi','Grigore Moisil Timisoara','Tudor Vladimirescu Tg Jiu',
           'Colegiul Informatica Piatra Neamt','Geoparcul Dinozaurilor Hateg','Constantin Brancoveanu Horezu',
           'Carmen Sylva Eforie','Magurele_BZ','Moroeni','Timisoara','Nicolae Bălcescu Oltenița'] # Names for the seismometers
# define the date-time for the start of the record
from datetime import date

today = date.today()
print("Today's date:", today.day-1)

zi=str(today.day-1)
print(zi)
an=str(today.year)
luna=str(today.month)
record_start=an +"-"+luna+"-"+zi+" 00:00:00"
dir_start=an+luna+zi
print(record_start)

# convert this into UTC Date Time format
starttime = UTCDateTime(record_start)
# The end time is one day later, there are 86400 seconds in a day
endtime = starttime + 86400
# read in list of Raspberry Shakes by looping through the list from the second seismometer
client=Client(base_url='https://fdsnws.raspberryshakedata.com/')
waveform = Stream()
# loop through the list of seismometers. The index for a list starts at 0 in Python
for x in range (0,len(seislist)):
    # add to the waveform with additional channels. If the station has no data, this will skip to the except option
    try:
        waveform+=client.get_waveforms('AM',seislist[x],'00','EHZ',starttime,endtime)
    except:
        print(seislist[x], "from", locations[x], "not returned from server, skipping this trace.")

waveform.merge(method=0, fill_value='latest')  

# place the mean value at zero on the axes
waveform.detrend(type='demean')

# comanda pentru filtrarea semnalului (local, regional, teleseism)
waveform.filter('bandpass', freqmin=0.1, freqmax=0.5, corners=2)

# automerge=False stops all of the traces with the same ID from being merged (and printed in a different order)
# method='full' allows zooming of the matplotlib graph when there are lots of data points, but slows down the performance
# equal_scale=False means that each subplot has a different y-axis scale.
# Change this to equal_scale=True if you want to see how amplitude decreases with distance.  
# print to screen, the code continues after you close this plot
waveform.plot(size=(1920,1080),type='normal', automerge=False, equal_scale=False, starttime=starttime, endtime=endtime)

#creare director cu numele de la record_start
# Directory 
#directory = dir_start
  
# Parent Directory path 
parent_dir = 'D:\\RoeduseisData\\2021\\Helicorder'
  
# Path 
path = os.path.join(parent_dir, dir_start) 
try:
    os.mkdir(path)
except FileExistsError:
    pass
  
# mode 
mode = 0o666

waveform[0].stats.station
for i in range(len(waveform)):
    #outfile = 'Date_arhiva/' + waveform[i].stats.station + '.png'
    outfile = path + '\\' + waveform[i].stats.station + '.png'
    
    #waveform[i].plot(type="dayplot", interval=60, right_vertical_labels=False, one_tick_per_line = True, color = ['k', 'r', 'b', 'g'], show_y_UTC_label=False, vertical_scaling_range=400, events={'min_magnitude': 6}, outfile=outfile, size=(1920,1920))
    #waveform[i].plot(type="dayplot", interval=60, right_vertical_labels=False, one_tick_per_line = True, color = ['k', 'r', 'b', 'g'], show_y_UTC_label=False, vertical_scaling_range=400, events={'min_magnitude': 6},size=(1200,800))
    
    waveform[i].plot(type="dayplot", interval=60, right_vertical_labels=False, one_tick_per_line = True, color = ['k', 'r', 'b', 'g'], show_y_UTC_label=False, vertical_scaling_range=400, outfile=outfile, size=(1920,1920))
    waveform[i].plot(type="dayplot", interval=60, right_vertical_labels=False, one_tick_per_line = True, color = ['k', 'r', 'b', 'g'], show_y_UTC_label=False, vertical_scaling_range=400, size=(1200,800))